# Data preparation
In questa sezione estrarremo i dati rilevanti per il clustering dei clienti.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("dataset/customer_supermarket.csv", sep="\t",index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471910 entries, 0 to 541909
Data columns (total 8 columns):
BasketID           471910 non-null object
BasketDate         471910 non-null object
Sale               471910 non-null object
CustomerID         406830 non-null float64
CustomerCountry    471910 non-null object
ProdID             471910 non-null object
ProdDescr          471157 non-null object
Qta                471910 non-null int64
dtypes: float64(1), int64(1), object(6)
memory usage: 32.4+ MB


Estraiamo le feature per ogni utente:
* I: the total number of items purchased by a customer during the period of
observation.

⚠️Si intende la somma delle quantità?

* Iu: the number of distinct items bought by a customer in the period of
observation.

* Imax: the maximum number of items purchased by a customer during a
shopping session

⚠️Si intende la quantità massima?

* E: the Shannon entropy on the purchasing behaviour of the customer

In [42]:
import numpy as np
def entropy(labels, base=None):
  """ Computes entropy of label distribution. """

  n_labels = len(labels)

  if n_labels <= 1:
    return 0

  value,counts = np.unique(labels, return_counts=True)
  probs = counts / n_labels
  n_classes = np.count_nonzero(probs)

  if n_classes <= 1:
    return 0

  ent = 0.

  # Compute entropy
  base = e if base is None else base
  for i in probs:
    ent -= i * np.math.log(i, base)

  return ent

In [44]:
for elem in df.CustomerID.unique():
    I=df[df["CustomerID"]==elem].Qta.sum()
    Iu=len(df[df["CustomerID"]==elem].ProdID.unique())
    Imax=df[df["CustomerID"]==elem].Qta.max()
    E=entropy(df[df["CustomerID"]==elem].ProdID,2)
    print("Customer id:\t"+str(elem))
    print("l:\t"+str(i))
    print("lu:\t"+str(iu))
    print("lmax:\t"+str(imax))
    


l:	3570
lu:	95
lmax:	9
Customer id:	13627.0
l:	3570
lu:	95
lmax:	9
Customer id:	12551.0
l:	3570
lu:	95
lmax:	9
Customer id:	12782.0
l:	3570
lu:	95
lmax:	9
Customer id:	14769.0
l:	3570
lu:	95
lmax:	9
Customer id:	15780.0
l:	3570
lu:	95
lmax:	9
Customer id:	12417.0
l:	3570
lu:	95
lmax:	9
Customer id:	17315.0
l:	3570
lu:	95
lmax:	9
Customer id:	16711.0
l:	3570
lu:	95
lmax:	9
Customer id:	12726.0
l:	3570
lu:	95
lmax:	9
Customer id:	15204.0
l:	3570
lu:	95
lmax:	9
Customer id:	17888.0
l:	3570
lu:	95
lmax:	9
Customer id:	17007.0
l:	3570
lu:	95
lmax:	9
Customer id:	15160.0
l:	3570
lu:	95
lmax:	9
Customer id:	14016.0
l:	3570
lu:	95
lmax:	9
Customer id:	13033.0
l:	3570
lu:	95
lmax:	9
Customer id:	12691.0
l:	3570
lu:	95
lmax:	9
Customer id:	13869.0
l:	3570
lu:	95
lmax:	9
Customer id:	14146.0
l:	3570
lu:	95
lmax:	9
Customer id:	12705.0
l:	3570
lu:	95
lmax:	9
Customer id:	16885.0
l:	3570
lu:	95
lmax:	9
Customer id:	17913.0
l:	3570
lu:	95
lmax:	9
Customer id:	12668.0
l:	3570
lu:	95
lmax:	9
Customer

KeyboardInterrupt: 

24

In [4]:
customer_dict = [ ('jack', 34, 'Sydeny' , 'Australia') ,
             ('Riti', 30, 'Delhi' , 'India' ) ,
             ('Vikas', 31, 'Mumbai' , 'India' ) ,
             ('Neelu', 32, 'Bangalore' , 'India' ) ,
             ('John', 16, 'New York' , 'US') ,
             ('Mike', 17, 'las vegas' , 'US')  ]
new_df = pd.DataFrame(customer_dict, columns = ['id','l' , 'lu', 'lmax' , 'E']) 

In [5]:
new_df.head()

,l,lu,lmax,E
0,jack,34,Sydeny,Australia
1,Riti,30,Delhi,India
2,Vikas,31,Mumbai,India
3,Neelu,32,Bangalore,India
4,John,16,New York,US
